# Reconnaissance de Pokemon par image

### Un peu de contexte

Le but de ce projet est de construire un réseau de neurones entièrement connecté capable de reconnaître un Pokemon à partir d'une image donnée.

L'implémentation est basée sur PyTorch et le modèle est entrainé et testé sur le dataset PokemonClassification : https://huggingface.co/datasets/keremberke/pokemon-classification.

In [10]:
import torch
from torch import nn
import copy
from torchvision.io import read_image
import numpy as np
from torchvision import transforms
from PIL import Image
import glob
import os
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
import matplotlib.pyplot as plt

### Étape 1 : Charger les données

En PyTorch, les données doivent être transmise au réseau de neurones à l'aide d'un loader. La première étape est de créer une classe de type `Dataset` que la fonction `DataLoader` prend en argument. La classe doit au moins posséder les trois routines `__init__`, `__len__` et `__getitem__`.

L'implémentation de cette classe est inspirée du tp1.a, avec une légère modification du `__getitem__` afin d'adapter les dimensions et que le tensor cible soit un scalaire.

Pour information, toutes les images chargées sont échantillonnées et réduites en niveaux de gris pour un soucis de rapidité sans en altérer la performance du model.

In [11]:
class CoRDataset(Dataset):
    def __init__(self, path, transform=None):
        self.imgs_path = path
        self.class_map = {}
        self.transform = transform
        file_list = glob.glob(self.imgs_path + "*")
        self.data=[]
        self.img =[]
        for i, class_path in enumerate(file_list):
            class_name = class_path.split("/")[-1]
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
                img = plt.imread(img_path)
                img = img/np.amax(img)
                R, G, B = img[::4,::4,0], img[::4,::4,1], img[::4,::4,2]
                img = 0.2989 * R + 0.5870 * G + 0.1140 * B # On passe l'image en niveaux de gris pour que ça soit plus rapide
                self.img.append(img)
            self.class_map[class_name] = i
        self.img_dim = (56, 56)    
    
    def __len__(self):
        return len(self.data)    
    
    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        class_id = self.class_map[class_name]
        img_tensor = torch.from_numpy(self.img[idx]).unsqueeze(0)  # Add channel dimension
        if self.transform:
            img_tensor = self.transform(img_tensor)
        return img_tensor.float(), class_id  # Return scalar class_id

### Étape 2 : Construire son réseau de neurone

Il existe deux méthodes pour créer son réseau de neurones :
- Créer un module, où les différentes couches et fonctions d'activations sont définies dans la routine `__init__` puis agancée dans la routine `__forward__` pour construire le réseau de neurone;
- Dans le cas d'un réseau de neurone classique, en utilisant la fonction `nn.Squential` qui prend comme argument la liste des couches et fonctions d'activations à enchaîner, dans le sens "forward".

Première méthode :

In [12]:
class PokemonModel(nn.Module):
    def __init__(self, num_classes):
        super(PokemonModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(p=0.5)
        self.flattened_size = 32 * 14 * 14  # Updated size after pooling
        self.fc1 = nn.Linear(self.flattened_size, 64)
        self.fc2 = nn.Linear(64, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.flattened_size)  # Flatten the tensor
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


pokemonmodel = PokemonModel(len(training_set.class_map))   
print(pokemonmodel)

PokemonModel(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=6272, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=110, bias=True)
)


Cette méthode devient utile si on veut "mettre les mains dans le cambouie". 

Seconde méthode : 

In [13]:
pokemonmodel = torch.nn.Sequential(torch.nn.Linear(56 * 56, 512),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(0.25),
                                       torch.nn.Linear(512, 256),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(0.25),
                                       torch.nn.Linear(256, len(training_set.class_map)),
                                       )

### Étape 3 :  choisir une fonction de coût et une méthode d'optimisation.

Ce n'est pas le sujet de ce TP. Dans notre cas nous prenons comme fonction de coût l'entreopie croisée binaire (BCE : *Binary Cross Entropy*) et pour l'optimisation, nous utilisons la descende de gradient stochastique (SGD : *Stochastic Gradient Descent*). Nous verrons au prochain cours pourquoi.

In [14]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pokemonmodel.parameters(), lr=0.001, weight_decay=10**(-3))

### Étape 4 : création des loader d'entraintement, de validation et de test

Ici on choisit un batch de 400, on discutera plus précisément du batch et du réglage de la taille du batch dans le chapitre 2.

In [15]:
BATCH_SIZE = 200
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(56, scale=(0.8, 1.0)),
    transforms.Normalize((0.5,), (0.5,))
])
if __name__ == "__main__":
    training_set = CoRDataset("data/training/", transform=transform)
    training_loader = DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True)
    validation_set = CoRDataset("data/validation/")
    validation_loader = DataLoader(validation_set, shuffle=False)
    test_set = CoRDataset("data/test/")
    test_loader = DataLoader(test_set, shuffle=False)

On définit ensuite les fonctions d'entrainement et de test `Learning` et `Testmodel`. Commentez ces deux fonctions afin de comprendre leur fonctionnement.

In [16]:
def Print_loss_accuracy(nepoch, tloss, vloss, accuracy, best_tloss, best_vloss, best_accuracy):
    print ("{:<6} {:<15} {:<17} {:<15} {:<20} {:<22} {:<15}".format(nepoch, tloss, vloss, accuracy, best_tloss, best_vloss, best_accuracy))

In [17]:
def Learning(nepoch, model, crit, optim, batchsize, trainingloader, validationloader):
    best_tloss = 100.
    best_vloss = 100.
    best_accuracy = 0.
    Print_loss_accuracy('Epoch', 'training loss', 'validation loss', 'accuracy', 'best train loss',
                        'best validation loss', 'best accuracy')

    for nepoch in range(nepoch):
        tloss = 0.
        vloss = 0.
        correct_test = 0
        model.train()

        for images, labels in trainingloader:
            optim.zero_grad()
            images = images.view(images.size(0), -1)  # Flatten the images
            predicted = model(images)
            loss = crit(predicted, labels)  # Ensure labels are not squeezed
            loss.backward()
            optim.step()
            tloss += loss.item() * images.size(0)

        tloss /= len(trainingloader.dataset)

        model.eval()

        with torch.no_grad():
            for images, labels in validationloader:
                images = images.view(images.size(0), -1)  # Flatten the images
                predicted = model(images)
                loss = crit(predicted, labels)
                correct_test += (predicted.argmax(1) == labels).sum().item()
                vloss += loss.item() * images.size(0)

        vloss /= len(validationloader.dataset)
        accuracy = 100 * correct_test / len(validationloader.dataset)

        if accuracy >= best_accuracy:
            torch.save(model, "best_model.pth")
            best_accuracy = accuracy
        if vloss <= best_vloss:
            best_vloss = vloss
        if tloss <= best_tloss:
            best_tloss = tloss

        Print_loss_accuracy(nepoch + 1,
                            np.round(tloss, 8),
                            np.round(vloss, 8),
                            np.round(accuracy, 8),
                            np.round(best_tloss, 8),
                            np.round(best_vloss, 8),
                            np.round(best_accuracy, 8))

In [18]:
def Testmodel(modelfile,crit, testloader):
    model = torch.load(modelfile)
    model.eval()
    plt.figure(dpi=300)
    ct=1
    for imgs, labels in testloader:
        image=imgs[0]
        plt.subplot(1, len(test_loader.sampler),ct)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        predicted = model(imgs.view(1,-1))
        test_loss = crit(predicted.squeeze(), labels.squeeze())
        plt.title('True label : {} \n Predicted label : {} \n Test loss : {}'.format(labels.squeeze().detach().numpy(),
                                                                       torch.squeeze(predicted).round().detach().numpy(),
                                                                                    np.round(test_loss.item(), 2)),
                  fontsize=6)
        ct += 1

### À vous de jouer !!

En reprenant l'exemple de cellule suivante, essayez de créer un réseau de neuronne entièrement connecté permettant d'identifier chats et lapins. Le model et la méthode d'optimisation doivent être réinitialiser à chaque fois. 

Vous pouvez rajouter des couches en intercallant la fonction d'activation linéaire rectifiée `nn.ReLU()` (ReLU : *Rectifide Linear Unit*) et couches linéaires `nn.Linear(size_in, size_out)`. 

Vous pouvez également augmenter le nombre d'epoch et regarder l'effet (ou non).

**Attention, le nombre d'entrées d'une couche doit être le nombre de sorties de la couche précédentes !**

In [ ]:
Learning(100, pokemonmodel, criterion, optimizer, BATCH_SIZE, training_loader, validation_loader)
Testmodel("best_model.pth", criterion, test_loader)

Epoch  training loss   validation loss   accuracy        best train loss      best validation loss   best accuracy  
1      4.65394612      5.03185767        0.0             4.65394612           5.03185767             0.0            
2      4.40825871      5.38519916        1.22302158      4.40825871           5.03185767             1.22302158     
3      4.24650124      5.63809341        1.43884892      4.24650124           5.03185767             1.43884892     
4      4.11095334      5.76879499        0.57553957      4.11095334           5.03185767             1.43884892     
5      4.01301803      5.83143888        1.29496403      4.01301803           5.03185767             1.43884892     
6      3.90334471      5.92381505        1.72661871      3.90334471           5.03185767             1.72661871     
7      3.79933683      5.95649987        2.08633094      3.79933683           5.03185767             2.08633094     
8      3.72930061      6.16343608        1.29496403      3.72930